# California housing prices 
.
Versão com o PySpark do kernel [California housing prices](https://github.com/robson-rsp/datascience/blob/main/regression/california_housing_prices.ipynb). Este notebook foi escrito no ambiente do Databricks e exportado como arquivo IPython .ipynb. Os textos daqui são cópias dos originais do link.
---
A tarefa neste kernel é criar um modelo de Machine Learning que estime preços de imóveis no estado da Califórnia baseado em características, como:

*   latitude/longitude: Localização do imóvel
*   housing_median_age: Idade média dos imóveis naquela quadra
*   totalRooms: Número total de cômodos dentro de um quarteirão
*   totalBedrooms: Número total de quartos dentro de um quarteirão
*   population: Número total de pessoas que moram em uma quadra
*   households: Número total de habitações em uma quadra
*   medianIncome: Renda média das famílias de uma quadra (medida em dezenas de milhares de dólares americanos)
*   medianHouseValue: Valor médio dos imóveis de uma quadra (medido em dólares americanos)
*   oceanProximity: Localização do imóvel em relação ao oceano/mar

Obs. A média populacional desses quarteirões é de 600 até 3000 pessoas.

Fonte: [Kaggle](https://www.kaggle.com/datasets/camnugent/california-housing-prices)

In [0]:

housing = spark.read.format("csv") \
          .option("inferSchema", "True") \
          .option("header", "True") \
          .option("sep", ",") \
          .load("/FileStore/tables/housing.csv")

# EDA

In [0]:
housing.sample(fraction=0.01).toPandas()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.27,37.85,52.0,1966.0,347.0,793.0,331.0,2.7750,152500.0,NEAR BAY
1,-122.26,37.80,20.0,2373.0,779.0,1659.0,676.0,1.6929,115000.0,NEAR BAY
2,-122.20,37.76,47.0,1116.0,259.0,826.0,279.0,1.7500,85700.0,NEAR BAY
3,-122.17,37.75,38.0,992.0,NaN,732.0,259.0,1.6196,85100.0,NEAR BAY
4,-122.28,37.86,52.0,3007.0,691.0,1582.0,636.0,2.5652,157700.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
209,-119.20,34.19,19.0,9503.0,1769.0,6370.0,1718.0,5.0016,218500.0,NEAR OCEAN
210,-118.95,34.17,9.0,2372.0,312.0,1039.0,321.0,7.6016,344900.0,<1H OCEAN
211,-118.81,34.25,4.0,9147.0,1827.0,3950.0,1661.0,5.7160,320800.0,<1H OCEAN
212,-121.55,38.55,10.0,6227.0,1164.0,2909.0,1077.0,4.1060,115900.0,INLAND


In [0]:
print(f'shape={housing.count(), len(housing.columns)}')

shape=(20640, 10)


In [0]:
housing.summary("count", "mean", "stddev", "min", "25%", "50%", "75%", "max").toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,25%,-121.8,33.93,18.0,1447.0,296.0,787.0,280.0,2.5625,119600.0,None
5,50%,-118.49,34.26,29.0,2127.0,435.0,1166.0,409.0,3.5347,179700.0,None
6,75%,-118.01,37.71,37.0,3146.0,647.0,1724.0,605.0,4.7426,264700.0,None
7,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


Verificando a ocorrência de campos NaN.

In [0]:
for name in housing.columns:
    mask = housing[name].isNull()
    print(f'{name}: {housing.where(mask).count()}')

longitude: 0
latitude: 0
housing_median_age: 0
total_rooms: 0
total_bedrooms: 207
population: 0
households: 0
median_income: 0
median_house_value: 0
ocean_proximity: 0


# Data visualization

Os gráficos foram criados utilizando as ferramentas de visualização nativas do Databricks. O resultado não pode ser visto em um arquivo .ipynb

# Train/Test split

In [0]:
train_set, test_set = housing.randomSplit([0.8, 0.2])

# Feature engineering

In [0]:
from pyspark.ml.feature import Imputer, StandardScaler, StringIndexer, VectorAssembler

## Categorical features

**Etapa ##:** Atributos nominais: Codificar

In [0]:
si = StringIndexer(inputCol='ocean_proximity', outputCol='ocean_proximity_')
si_model = si.fit(train_set)
train_set = si_model.transform(train_set)
test_set  = si_model.transform(test_set)

## Numerical features

**Etapa ##:** NaN values: Imputar

In [0]:
im = Imputer(strategy='mean', inputCol='total_bedrooms', outputCol='total_bedrooms_')
im_model = im.fit(train_set)
train_set = im_model.transform(train_set)
test_set  = im_model.transform(test_set)

**Etapa ##:** Os atributos 'total_rooms' e 'total_bedrooms' não fazem sentido se eu deixá-los da forma como estão. 'total_rooms' refere-se à quantidade total de cômodos que há em cada quadra. 'total_bedrooms' refere-se à quantidade total de quartos que há em cada quadra. Vou criar novos atributos a partir dos existentes. Vou fazer isso dentro de uma classe para usar a classe Pipeline.
1. **mean_bedrooms_households:** Quantidade média de quartos por habitação.
1. **mean_rooms_households:** Quantidade média de cômodos por habitação.
1. **mean_population_households:** Quantidade média de moradores por imóvel.

In [0]:
train_set = train_set.withColumn('mean_bedrooms_households', train_set['total_bedrooms_'] / train_set['households'])
train_set = train_set.withColumn('mean_rooms_households', train_set['total_rooms'] / train_set['households'])
train_set = train_set.withColumn('mean_population_households', train_set['population'] / train_set['households'])

test_set = test_set.withColumn('mean_bedrooms_households', test_set['total_bedrooms_'] / test_set['households'])
test_set = test_set.withColumn('mean_rooms_households', test_set['total_rooms'] / test_set['households'])
test_set = test_set.withColumn('mean_population_households', test_set['population'] / test_set['households'])

**Etapa ##:** Unir todas as colunas em uma única que será utilizada pelos modelos de Machine Learning do PySpark. Essa etapa dever ser feita antes da padronização dos atributos com StandardScaler.

In [0]:
cols = ['longitude', 'latitude', 'housing_median_age', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity_', 'total_bedrooms_', 'mean_bedrooms_households', 'mean_rooms_households', 'mean_population_households']

va = VectorAssembler(inputCols=cols, outputCol='X')
train_set = va.transform(train_set)
test_set  = va.transform(test_set)

**Etapa ##:** Padronização dos atributos com StandardScaler.

In [0]:
ss = StandardScaler(inputCol='X', outputCol='X_')
ss_model = ss.fit(train_set)
train_set = ss_model.transform(train_set)
test_set  = ss_model.transform(test_set)

**Etapa ##:** Selecionar X e y dos conjuntos de treino e teste. Vou renomear "X_" para "X"  e "median_house_value" para "y" assim os conjuntos ficarão de acordo com o padrão de nomenclatura.

In [0]:
train_set = train_set.select('X_', 'median_house_value')
test_set  = test_set.select('X_', 'median_house_value')

train_set = train_set.withColumnRenamed(existing='X_', new='X')
test_set  = test_set.withColumnRenamed(existing='X_', new='X')

train_set = train_set.withColumnRenamed(existing='median_house_value', new='y')
test_set  = test_set.withColumnRenamed(existing='median_house_value', new='y')

# Model training


In [0]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

## ElasticNet

In [0]:
lr = LinearRegression(featuresCol='X', labelCol='y', elasticNetParam=0.0)
lr_model = lr.fit(train_set)
y_pred = lr_model.transform(test_set)

re = RegressionEvaluator(labelCol='y', metricName='mse')
print(f'mse: {re.evaluate(y_pred)}')
y_pred.show()

mse: 1.0080698722616673e-13
+--------------------+--------+------------------+
|                   X|       y|        prediction|
+--------------------+--------+------------------+
|[-61.971207918053...| 85800.0|  85800.0000000563|
|[-61.971207918053...|103600.0|103600.00000008664|
|[-61.936308605469...| 50800.0|  50799.9999997033|
|[-61.931322989385...| 78300.0| 78300.00000013468|
|[-61.911380525051...| 72200.0| 72199.99999976232|
|[-61.911380525051...| 70200.0| 70199.99999977398|
|[-61.911380525051...| 64600.0| 64599.99999974954|
|[-61.906394908968...| 81300.0| 81299.99999973521|
|[-61.906394908968...| 70500.0| 70499.99999979833|
|[-61.906394908968...| 60000.0| 59999.99999987355|
|[-61.901409292884...| 85100.0| 85099.99999973117|
|[-61.901409292884...| 82000.0| 81999.99999976349|
|[-61.901409292884...| 81800.0| 81799.99999979051|
|[-61.896423676801...|104200.0|104199.99999979085|
|[-61.896423676801...|103100.0|103100.00000025943|
|[-61.891438060717...| 92500.0| 92499.99999947792|
|[-

## RandomForestRegressor

In [0]:
rf = RandomForestRegressor(featuresCol='X', labelCol='y')
rf_model = rf.fit(train_set)
y_pred = rf_model.transform(test_set)

re = RegressionEvaluator(labelCol='y', metricName='mse')
print(f'mse: {re.evaluate(y_pred)}')
y_pred.show()

mse: 620236500.7274466
+--------------------+--------+------------------+
|                   X|       y|        prediction|
+--------------------+--------+------------------+
|[-61.971207918053...| 85800.0| 89026.55630848241|
|[-61.971207918053...|103600.0| 112057.5689520153|
|[-61.936308605469...| 50800.0| 114857.1023909621|
|[-61.931322989385...| 78300.0| 86064.29282864387|
|[-61.911380525051...| 72200.0| 86653.09945847519|
|[-61.911380525051...| 70200.0| 86653.09945847519|
|[-61.911380525051...| 64600.0| 83028.70534665466|
|[-61.906394908968...| 81300.0| 91676.10041269015|
|[-61.906394908968...| 70500.0|   84835.326913311|
|[-61.906394908968...| 60000.0| 86668.84994405488|
|[-61.901409292884...| 85100.0| 96785.77599298744|
|[-61.901409292884...| 82000.0|103447.34013702514|
|[-61.901409292884...| 81800.0|115595.60806046604|
|[-61.896423676801...|104200.0|114797.11701733459|
|[-61.896423676801...|103100.0| 92655.82809429971|
|[-61.891438060717...| 92500.0|104054.71960671115|
|[-61.89